In [36]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

# <span style="color:red"><<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>></span>

In [22]:
!pip install pyspark==2.4.5

In [23]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [24]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession.builder.getOrCreate()

In [25]:
# clone the dataset from github
!git clone https://github.com/wchill/HMP_Dataset.git

fatal: destination path 'HMP_Dataset' already exists and is not an empty directory.


In [26]:
# look at the file list in HMP_Dataset
! ls HMP_Dataset

Brush_teeth	Drink_glass  Getup_bed	  Pour_water	 Use_telephone
Climb_stairs	Eat_meat     impdata.py   README.txt	 Walk
Comb_hair	Eat_soup     Liedown_bed  Sitdown_chair
Descend_stairs	final.py     MANUAL.txt   Standup_chair


In [27]:
# look at whats inside Brush_teeth folder
! ls HMP_Dataset/Brush_teeth

Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt
Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt
Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt
Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt
Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt
Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt
Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt
Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt


In [28]:
# look at the txt file
! cat HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt

22 49 35
22 49 35
22 52 35
22 52 35
21 52 34
22 51 34
20 50 35
22 52 34
22 50 34
22 51 35
21 51 33
20 50 34
21 49 33
21 49 33
20 51 35
18 49 34
19 48 34
16 53 34
18 52 35
18 51 32
19 50 33
19 53 33
21 50 33
21 51 33
24 51 34
25 50 35
25 51 36
25 50 38
25 51 39
26 49 40
25 49 41
25 47 42
22 45 41
21 48 41
21 48 43
21 47 41
22 47 41
21 47 42
22 47 42
22 47 42
24 48 42
20 49 41
22 49 40
24 47 38
22 49 37
24 49 36
24 50 36
23 49 37
24 50 37
23 49 36
23 52 36
23 51 37
23 50 37
24 50 38
21 52 37
22 51 38
22 52 38
24 52 39
23 50 39
22 50 39
22 53 37
23 51 39
23 50 39
23 50 39
22 50 38
22 50 37
23 49 38
23 50 38
23 49 37
22 51 38
22 51 38
22 50 38
22 49 38
23 49 37
22 50 37
22 49 38
22 49 38
23 52 38
23 51 39
22 50 38
23 49 38
23 49 38
22 51 38
23 50 38
23 50 38
23 50 37
22 49 38
24 50 38
22 51 37
23 50 40
23 50 38
22 51 38
23 50 37
23 50 39
22 50 38
24 51 37
23 49 37
23 50 37
22 51 37
22 49 37
23 49 37
24 49 37
22 51 38
23 51 38
22 51 38
22 49 38
20 51 38
20 49 37
21 50 37
22 49 38
23 49 37
2

In [29]:
# define the schema
# since the txt files have 3 columns, we define the schema to have 3 fields
from pyspark.sql.types import StructType,StructField,IntegerType

schema_HMP = StructType([StructField('x',IntegerType(),True), StructField('y',IntegerType(),True), StructField('z',IntegerType(),True)])

In [30]:
# import operation system
import os

In [31]:
# look whats under the HMP_Dataset direction with file type 
# and save the file list in to a list
file_list = os.listdir('HMP_Dataset')
file_list

['Pour_water',
 'impdata.py',
 'Drink_glass',
 'Use_telephone',
 'MANUAL.txt',
 'Sitdown_chair',
 'Walk',
 'Standup_chair',
 'Climb_stairs',
 'README.txt',
 'Liedown_bed',
 'Brush_teeth',
 'Eat_soup',
 'Comb_hair',
 'Eat_meat',
 'final.py',
 'Descend_stairs',
 '.idea',
 'Getup_bed',
 '.git']

In [32]:
file_list_filtered = [s for s in file_list if '_' in s]
file_list_filtered

['Pour_water',
 'Drink_glass',
 'Use_telephone',
 'Sitdown_chair',
 'Standup_chair',
 'Climb_stairs',
 'Liedown_bed',
 'Brush_teeth',
 'Eat_soup',
 'Comb_hair',
 'Eat_meat',
 'Descend_stairs',
 'Getup_bed']

In [33]:
# reading the files and put them into a DataFrame
df = None
dfs = []
# lit adds literal or constant to Spark DataFrame
from pyspark.sql.functions import lit

for folder in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/'+ folder)
    print('reading under '+ folder + ' folder')
    
    for data_file in data_files:
        print('reading file:', data_file)
        
        # read the txt file in to a temporary DataFrame and add two columns
        file_dir = 'HMP_Dataset/'+ folder + '/'+ data_file
        temp_df = spark.read.option('header','true').option('header','false').option('delimiter', ' ').csv(file_dir,schema = schema_HMP)
        temp_df = temp_df.withColumn("class",lit(folder))
        temp_df = temp_df.withColumn("source",lit(data_file))
        
        # append the temporary DataFrame to the DataFrame we defined vertically
        if df is None:
            df = temp_df 
        else:
            df = df.union(temp_df)

reading under Pour_water folder
reading file: Accelerometer-2011-06-06-09-55-25-pour_water-f1.txt
reading file: Accelerometer-2012-06-09-22-55-08-pour_water-m11.txt
reading file: Accelerometer-2011-06-02-16-52-30-pour_water-f4.txt
reading file: Accelerometer-2012-05-29-17-19-55-pour_water-m3.txt
reading file: Accelerometer-2012-06-09-22-55-42-pour_water-m11.txt
reading file: Accelerometer-2011-06-01-14-27-40-pour_water-f1.txt
reading file: Accelerometer-2012-06-09-22-56-17-pour_water-m11.txt
reading file: Accelerometer-2011-06-06-09-44-12-pour_water-f1.txt
reading file: Accelerometer-2011-05-30-21-51-05-pour_water-m2.txt
reading file: Accelerometer-2012-05-30-19-07-14-pour_water-m4.txt
reading file: Accelerometer-2012-05-29-17-12-07-pour_water-m3.txt
reading file: Accelerometer-2012-06-07-21-31-24-pour_water-f4.txt
reading file: Accelerometer-2012-06-07-21-35-50-pour_water-f4.txt
reading file: Accelerometer-2011-06-06-09-56-28-pour_water-f1.txt
reading file: Accelerometer-2012-05-30-19

reading file: Accelerometer-2012-05-30-19-53-49-drink_glass-f2.txt
reading file: Accelerometer-2012-03-26-04-53-07-drink_glass-f2.txt
reading file: Accelerometer-2011-06-01-14-15-19-drink_glass-f1.txt
reading file: Accelerometer-2011-06-02-16-51-58-drink_glass-f4.txt
reading file: Accelerometer-2011-06-02-17-20-17-drink_glass-m1.txt
reading file: Accelerometer-2011-03-24-16-08-10-drink_glass-f2.txt
reading file: Accelerometer-2011-06-01-16-49-14-drink_glass-f1.txt
reading file: Accelerometer-2011-06-01-15-25-43-drink_glass-f1.txt
reading file: Accelerometer-2011-06-01-15-09-37-drink_glass-f1.txt
reading file: Accelerometer-2011-06-02-17-20-47-drink_glass-m1.txt
reading file: Accelerometer-2012-05-30-19-58-44-drink_glass-m3.txt
reading file: Accelerometer-2011-05-30-21-06-42-drink_glass-f1.txt
reading file: Accelerometer-2012-03-23-03-47-02-drink_glass-m9.txt
reading file: Accelerometer-2012-05-29-17-11-01-drink_glass-m3.txt
reading file: Accelerometer-2011-06-02-17-40-40-drink_glass-m1

reading file: Accelerometer-2011-06-02-17-25-34-sitdown_chair-m1.txt
reading file: Accelerometer-2011-06-06-09-35-11-sitdown_chair-f1.txt
reading file: Accelerometer-2011-04-08-17-33-10-sitdown_chair-f3.txt
reading file: Accelerometer-2011-05-30-20-48-57-sitdown_chair-f1.txt
reading file: Accelerometer-2011-05-30-20-55-28-sitdown_chair-f1.txt
reading file: Accelerometer-2011-05-31-16-27-00-sitdown_chair-f1.txt
reading file: Accelerometer-2011-05-31-14-53-37-sitdown_chair-f1.txt
reading file: Accelerometer-2011-06-02-17-44-20-sitdown_chair-m1.txt
reading file: Accelerometer-2011-12-11-08-24-32-sitdown_chair-m2.txt
reading file: Accelerometer-2011-06-02-16-43-56-sitdown_chair-f4.txt
reading file: Accelerometer-2011-12-05-09-50-54-sitdown_chair-f1.txt
reading file: Accelerometer-2011-05-30-08-31-30-sitdown_chair-f1.txt
reading file: Accelerometer-2011-06-02-17-32-07-sitdown_chair-m1.txt
reading file: Accelerometer-2011-05-30-21-04-13-sitdown_chair-f1.txt
reading file: Accelerometer-2011-1

reading file: Accelerometer-2011-03-24-16-09-19-standup_chair-f2.txt
reading file: Accelerometer-2012-05-25-18-27-29-standup_chair-f4.txt
reading file: Accelerometer-2011-12-11-08-12-46-standup_chair-f4.txt
reading file: Accelerometer-2011-06-02-17-31-39-standup_chair-m1.txt
reading file: Accelerometer-2011-12-05-09-48-17-standup_chair-f1.txt
reading file: Accelerometer-2011-06-01-14-39-38-standup_chair-f1.txt
reading file: Accelerometer-2011-06-02-17-26-02-standup_chair-m1.txt
reading file: Accelerometer-2012-05-29-17-12-51-standup_chair-m3.txt
reading file: Accelerometer-2011-06-01-14-52-02-standup_chair-f1.txt
reading file: Accelerometer-2011-04-05-18-25-12-standup_chair-f1.txt
reading file: Accelerometer-2011-12-11-08-13-41-standup_chair-f4.txt
reading file: Accelerometer-2011-03-24-11-19-29-standup_chair-f1.txt
reading file: Accelerometer-2011-06-02-17-42-24-standup_chair-m1.txt
reading file: Accelerometer-2011-12-11-08-24-09-standup_chair-m2.txt
reading file: Accelerometer-2011-0

reading file: Accelerometer-2012-05-30-18-32-56-climb_stairs-f3.txt
reading file: Accelerometer-2012-06-06-09-43-47-climb_stairs-m6.txt
reading file: Accelerometer-2012-06-06-14-13-20-climb_stairs-m7.txt
reading file: Accelerometer-2012-05-30-22-07-15-climb_stairs-m2.txt
reading file: Accelerometer-2011-04-05-18-21-22-climb_stairs-f1.txt
reading file: Accelerometer-2012-05-30-18-31-08-climb_stairs-f3.txt
reading file: Accelerometer-2012-06-06-09-37-58-climb_stairs-m6.txt
reading file: Accelerometer-2012-06-06-14-05-05-climb_stairs-m7.txt
reading file: Accelerometer-2012-05-30-22-06-17-climb_stairs-m2.txt
reading file: Accelerometer-2012-05-28-17-56-03-climb_stairs-m1.txt
reading file: Accelerometer-2011-06-02-10-37-44-climb_stairs-f1.txt
reading file: Accelerometer-2012-06-07-10-53-34-climb_stairs-f1.txt
reading file: Accelerometer-2012-06-06-08-54-29-climb_stairs-m5.txt
reading file: Accelerometer-2012-06-07-10-49-35-climb_stairs-f1.txt
reading file: Accelerometer-2012-05-30-18-33-57-

reading file: Accelerometer-2011-05-30-21-09-46-descend_stairs-f1.txt
reading file: Accelerometer-2011-04-11-11-51-27-descend_stairs-f1.txt
reading file: Accelerometer-2011-05-30-10-19-56-descend_stairs-m1.txt
reading file: Accelerometer-2011-05-30-10-37-30-descend_stairs-m1.txt
reading file: Accelerometer-2011-06-02-10-36-42-descend_stairs-f1.txt
reading file: Accelerometer-2011-05-30-21-39-11-descend_stairs-m2.txt
reading file: Accelerometer-2011-05-30-08-34-06-descend_stairs-f1.txt
reading file: Accelerometer-2011-03-29-16-17-18-descend_stairs-f1.txt
reading file: Accelerometer-2011-06-06-10-56-23-descend_stairs-f1.txt
reading file: Accelerometer-2011-03-29-16-16-34-descend_stairs-f1.txt
reading file: Accelerometer-2011-05-31-16-28-22-descend_stairs-f1.txt
reading file: Accelerometer-2011-06-06-10-58-40-descend_stairs-f1.txt
reading file: Accelerometer-2011-05-30-21-32-18-descend_stairs-m2.txt
reading file: Accelerometer-2011-03-29-16-18-14-descend_stairs-f1.txt
reading file: Accele

reading file: Accelerometer-2011-05-31-14-57-24-getup_bed-f1.txt
reading file: Accelerometer-2012-06-12-15-35-29-getup_bed-m4.txt
reading file: Accelerometer-2011-03-29-09-21-17-getup_bed-f1.txt
reading file: Accelerometer-2012-06-20-21-53-06-getup_bed-m10.txt
reading file: Accelerometer-2012-06-07-22-20-59-getup_bed-f4.txt
reading file: Accelerometer-2012-06-07-22-24-33-getup_bed-f4.txt
reading file: Accelerometer-2012-06-07-22-17-45-getup_bed-f4.txt
reading file: Accelerometer-2012-06-07-22-19-25-getup_bed-f4.txt
reading file: Accelerometer-2011-06-02-16-57-57-getup_bed-f4.txt
reading file: Accelerometer-2012-06-20-20-30-45-getup_bed-f5.txt


In [37]:
# index the classes
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = 'class', outputCol = 'class_Index')
indexed_df = indexer.fit(df).transform(df)

indexed_df.show()

+---+---+---+----------+--------------------+-----------+
|  x|  y|  z|     class|              source|class_Index|
+---+---+---+----------+--------------------+-----------+
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|
| 29| 41| 50|Pour_water|Accelerometer-201...|        2.0|
| 29| 41| 50|Pour_water|Accelerometer-201...|        2.0|
| 30| 41| 51|Pour_water|Accelerometer-201...|        2.0|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|
| 28| 41| 50|Pour_water|Accelerometer-201...|        2.0|
| 29| 42| 50|Pour_water|Accelerometer-201...|        2.0|
| 28| 43| 51|Pour_water|Accelerometer-201...|        2.0|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|
| 29| 42| 51|P

In [38]:
# check how many classes the df has
indexed_df.groupBy("class").count().show()

+--------------+-----+
|         class|count|
+--------------+-----+
| Use_telephone|15225|
| Standup_chair|25417|
|      Eat_meat|31236|
|     Getup_bed|45801|
|   Drink_glass|42792|
|    Pour_water|41673|
|     Comb_hair|23504|
|  Climb_stairs|40258|
| Sitdown_chair|25036|
|   Liedown_bed|11446|
|Descend_stairs|15375|
|   Brush_teeth|29829|
|      Eat_soup| 6683|
+--------------+-----+



In [39]:
# one hot endcoding
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol = 'class_Index', outputCol = 'categoryVec')
encoded_df = encoder.transform(indexed_df)

encoded_df.show()

# the last column is the Apache Spark representation of sparse vector
# in the fisrt element of the last column, 12 is the size of the vector, 0 is the index of the vector where the data is not null, [1.0] is the value at the index
# e.g. SparseVector(4, [1, 3], [3.0, 4.0]) = [0,3,0,4]

+---+---+---+----------+--------------------+-----------+--------------+
|  x|  y|  z|     class|              source|class_Index|   categoryVec|
+---+---+---+----------+--------------------+-----------+--------------+
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 29| 41| 50|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 29| 41| 50|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 30| 41| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|
| 28| 41| 50|Pour_water|Accelerometer-201...|      

In [40]:
# transform x,y,z into vectors since sparkML can ONLY work with VECTORS
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['x','y','z'], outputCol = 'feature_vector')
vectorized_df = vectorAssembler.transform(encoded_df)

vectorized_df.show()

+---+---+---+----------+--------------------+-----------+--------------+----------------+
|  x|  y|  z|     class|              source|class_Index|   categoryVec|  feature_vector|
+---+---+---+----------+--------------------+-----------+--------------+----------------+
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[28.0,42.0,51.0]|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[28.0,42.0,51.0]|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|
| 29| 41| 50|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[29.0,41.0,50.0]|
| 29| 41| 50|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[29.0,41.0,50.0]|
| 30| 41| 

In [41]:
# normalize features
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol = 'feature_vector', outputCol = 'normalized_vector',p=1.0)
normalized_df = normalizer.transform(vectorized_df)

normalized_df.show()

+---+---+---+----------+--------------------+-----------+--------------+----------------+--------------------+
|  x|  y|  z|     class|              source|class_Index|   categoryVec|  feature_vector|   normalized_vector|
+---+---+---+----------+--------------------+-----------+--------------+----------------+--------------------+
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
|

In [43]:
# create pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [indexer,encoder,vectorAssembler,normalizer])
# this is not a complete ML model yet but later we will do ML algos
model = pipeline.fit(df)

prediction = model.transform(df)


In [44]:
prediction.show()

+---+---+---+----------+--------------------+-----------+--------------+----------------+--------------------+
|  x|  y|  z|     class|              source|class_Index|   categoryVec|  feature_vector|   normalized_vector|
+---+---+---+----------+--------------------+-----------+--------------+----------------+--------------------+
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
| 28| 42| 51|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
| 55| 28| 33|Pour_water|Accelerometer-201...|        2.0|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
|

In [46]:
# get rid of columns we don't need
df_train = prediction.drop('x').drop('y').drop('z').drop('class').drop('source').drop('fetures').drop('class_Index')

df_train.show()

+--------------+----------------+--------------------+
|   categoryVec|  feature_vector|   normalized_vector|
+--------------+----------------+--------------------+
|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
|(12,[2],[1.0])|[55.0,28.0,33.0]|[0.47413793103448...|
|(12,[2],[1.0])|[29.0,41.0,50.0]|[0.24166666666666...|
|(12,[2],[1.0])|[29.0,41.0,50.0]|[0.24166666666666...|
|(12,[2],[1.0])|[30.0,41.0,51.0]|[0.24590163934426...|
|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
|(12,[2],[1.0])|[28.0,41.0,50.0]|[0.23529411764705...|
|(12,[2],[1.0])|[29.0,42.0,50.0]|[0.23966942148760...|
|(12,[2],[1.0])|[28.0,43.0,51.0]|[0.22950819672131...|
|(12,[2],[1.0])|[28.0,42.0,51.0]|[0.23140495867768...|
|(12,[2],[1.0])|[29.0,42.0,51.0]|[0.23770491803278...|
|(12,[2],[